In [2]:
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')
from utils import ChatOpenAI

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

In [3]:
from langchain.agents import Tool, load_tools
import math
import pandas as pd
from tqdm import tqdm
from langchain.agents import initialize_agent, AgentType
from langchain.chains import LLMMathChain
import warnings
warnings.filterwarnings('ignore')

In [4]:
# _______________________________________________ЗАДАНИЕ 3.3.9________________________________________________________________



df1 = pd.read_csv("https://stepik.org/media/attachments/lesson/1110884/questions.csv")
df1

,texts,answers
0,Человек использует только 10% своего мозга.,?
1,Кошки и собаки видят мир в черно-белых тонах.,?
2,Дейтерий - это изотоп водорода.,?
3,Пчёлы не могут видеть красный цвет.,?
4,Нейтроны - это элементарные частицы с отрицате...,?
5,Земля обращается вокруг Солнца по эллиптическо...,?
6,"Период полураспада урана-238 составляет 4,5 ми...",?
7,"""Дельта"" в науке обозначает изменение квантово...",?
8,Антибиотики эффективны против вирусов.,?
9,Все диамагнетики являются суперпроводниками.,?


In [5]:
llm_math = LLMMathChain.from_llm(llm=llm)

# создаём math tool, задаём имя, описание и функцию
math_tool = Tool(
    name='Калькулятор',
    func=llm_math.run,
    description='Может производить математические расчёты.'
)

# Для передачи в агента, нам нужно поместить инструменты в список
tools = [math_tool]

# создаём LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm.predict,
    description='Отвечает на общие вопросы'
)

# Добавляем его к другим инструментам
tools.append(llm_tool)
tools = load_tools(["human"])

# Создадим нашего первого агента, передав в него инструменты 
first_agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3
)

In [7]:
answers = []

for text in tqdm(df1['texts']):


    try:

        # Попытка обработки задачи агентом

        result = first_agent.run(f'{text}. Ответ должен быть представлен в виде булева значения True или False без ковычек. Если ты хоть немного сомневаешься то отвечай булевым значением False. Если нет ответа спроси')
        #  Не допускаются в ответе содержания слов и ответов типа:\'True\',\'False\' 1,0, да, нет, ложь, истина, неверно, верно.
        print(f"Задача: {text}\nРезультат: {result}\n")  # Выводим успешный результат
        # break # Для отладки. Уберите, когда убедитесь, что на одном примере работает
    except Exception as e:

        # Сохранение ошибки

        result = f"Ошибка: {e}"

        print(f"Задача: {text}\nРезультат: {result}\n")  # Выводим ошибку
       
    answers.append(result)

 

# Добавление столбца с ответами

df1['answers'] = answers
# Сохранение в CSV

df1[['texts', 'answers']].to_csv("3.3.9_solution.csv", index=False) 


  0%|          | 0/25 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...


  4%|▍         | 1/25 [00:01<00:37,  1.58s/it]

Thought: Я знаю, что утверждение о том, что человек использует только 10% своего мозга, является мифом. Научные исследования показывают, что мы используем все части нашего мозга, хотя не все одновременно. Поэтому я могу с уверенностью ответить на этот вопрос.

Action:
```
{
  "action": "Final Answer",
  "action_input": "False"
}
```


> Finished chain.
Задача: Человек использует только 10% своего мозга.
Результат: False



> Entering new AgentExecutor chain...


  8%|▊         | 2/25 [00:02<00:33,  1.44s/it]

...

Thought: Я не уверен, размещён ли курс на Stepik, так как у меня нет доступа к текущей информации о курсах. Поэтому я должен ответить False. 
Action:
```
{
  "action": "Final Answer",
  "action_input": "False"
}
```


> Finished chain.
Задача: Курс который вы сейчас проходите размещён на Stepik.
Результат: False



> Entering new AgentExecutor chain...


 96%|█████████▌| 24/25 [00:50<00:02,  2.20s/it]

Thought: Я не знаю, кто является преподавателем курса, поэтому не могу подтвердить или опровергнуть информацию о Валере Бабушкине. Поскольку я не уверен, я должен ответить False. 
Action:
```
{
  "action": "Final Answer",
  "action_input": "False"
}
```


> Finished chain.
Задача: Одного из преподавателей этого курса зовут Валера Бабушкин.
Результат: False



> Entering new AgentExecutor chain...


100%|██████████| 25/25 [00:53<00:00,  2.14s/it]

Thought: Пушкин родился в 1799 году, а не в 1837, и не в Нью-Йорке, а в Москве. Поэтому утверждение неверно. 
Action:
```
{
  "action": "Final Answer",
  "action_input": "False"
}
```


> Finished chain.
Задача: Пушкин родился в 1837 году в Нью-Йорке.
Результат: False

